# Approximate-Nearest-Neighbors Image Search Pipeline

This notebook details a simple pipeline for performing approximate nearest neighbors search on a dataset of images by tokenizing the images into vectors using a pre-trained model and then using a standard library for approximate nearest neighbors search. The metric used for the search is cosine similarity.

We consider the following implementation a baseline for image search results, utilizing simple yet powerful tools to achieve a good performance. 

In [2]:
import os
from PIL import Image
import torch
from torchvision import transforms
from transformers import CLIPModel, CLIPProcessor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import faiss
import time

/Users/naomi/miniconda3/envs/Lab2_env/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/naomi/miniconda3/envs/Lab2_env/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <5AA8DD3D-A2CC-31CA-8060-88B4E9C18B09> /Users/naomi/miniconda3/envs/Lab2_env/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <CDAC6E34-8608-3E70-8B2F-32BCD38E90FB> /Users/naomi/miniconda3/envs/Lab2_env/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Data

### Load Dataset

In [3]:
# Set up the folder paths
dataset_path = "datasets/house_styles"
image_folder = "datasets/house_styles/all_images"
label_file = "datasets/house_styles/labels.csv"
split_file = "datasets/house_styles/split_mask.csv"

# Output tokenized database
output_index_file = "vector_dbs/house_styles/image_vectors_index.npy"
output_query_file = "vector_dbs/house_styles/image_vectors_query.npy"

### Load Computer Vision Model

In [ ]:
# Load the pretrained CLIP model and processor from Hugging Face
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Set up the image transformation pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
])


### Tokenize Images

In [ ]:
### Train - Test Split: Run Only Once ###

image_names = [f for f in os.listdir(image_folder)]
print(f"Found {len(image_names)} images in '{image_folder}'.")

random_seed = 42
train_ratio = 0.8

# Randomly split the image names into training and testing sets
train_names, test_names = train_test_split(
    image_names,
    train_size=train_ratio,
    random_state=random_seed,
    shuffle=True
)

# Create a DataFrame to store image names and their corresponding split
split_df = pd.DataFrame({
    'image_name': image_names,
    'split': ['train' if name in train_names else 'test' for name in image_names]
})

# Save the split information to a CSV file
split_df.to_csv(split_file, index=False)

In [22]:
# Load the split information from the CSV file
split_df = pd.read_csv(split_file)

# Extract training image names
index_image_names = split_df[split_df['split'] == 'train']['image_name'].tolist()
query_image_names = split_df[split_df['split'] == 'test']['image_name'].tolist()
all_image_names = index_image_names + query_image_names

print(f"Number of index images to process: {len(index_image_names)}")
print(f"Number of query images to process: {len(query_image_names)}")


Number of index images to process: 693
Number of query images to process: 174


In [46]:
# Function to load and preprocess an image
def load_and_preprocess_image(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        return transform(image)
    except Exception as e:
        raise IOError(f"Error loading image '{image_path}': {e}")

# Function to tokenize and get vector representation of an image
def get_image_vector(image_tensor):
    # Normalize after conversion to avoid PIL conversion issues
    # normalized_tensor = transforms.Normalize(
    #     mean=(0.48145466, 0.4578275, 0.40821073),
    #     std=(0.26862954, 0.26130258, 0.27577711)
    # )(image_tensor)

    # Add a batch dimension
    inputs = processor(images=image_tensor, return_tensors="pt")
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    return image_features.cpu().numpy().flatten()

def save_image_vectors(image_names, output_file):
    image_vectors = []
    for image_name in tqdm(image_names):
        image_path = os.path.join(image_folder, image_name)
        try:
            image_tensor = load_and_preprocess_image(image_path)
            image_vector = get_image_vector(image_tensor)
            image_vectors.append(image_vector)
        except Exception as e:
            print(f"Error processing image '{image_path}': {e}")

    image_vectors = np.stack(image_vectors)
    np.save(output_file, image_vectors)
    return image_vectors

In [49]:
index_vectors = save_image_vectors(index_image_names, output_index_file)
print(f"{len(index_vectors)} image vectors saved to '{output_index_file}'.")

query_vectors = save_image_vectors(query_image_names, output_query_file)
print(f"{len(query_vectors)} image vectors saved to '{output_query_file}'.")

100%|██████████| 693/693 [00:48<00:00, 14.37it/s]


693 image vectors saved to 'vector_dbs/house_styles/image_vectors_index.npy'.


100%|██████████| 174/174 [00:11<00:00, 15.09it/s]

174 image vectors saved to 'vector_dbs/house_styles/image_vectors_query.npy'.


### Load Tokenized Images

In [4]:
index_vectors = np.load(output_index_file)
query_vectors = np.load(output_query_file)

print(f"Loaded {len(index_vectors)} index vectors and {len(query_vectors)} query vectors.")


Loaded 693 index vectors and 174 query vectors.


## Ground Truth

In [23]:
def semi_optimized_exhaustive_search(
        index_vectors: np.ndarray,
        query_vectors: np.ndarray,
        k: int,
):
    """
    This function performs an optimized exhaustive search.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        dim: The dimensionality of the vectors.
    Returns:
        An array of shape (n_queries, k) containing the indices of the k nearest neighbors for each query vector.
    """
    ann_lists = []
    for query_vec in tqdm(query_vectors):
        # distances = np.linalg.norm(index_vectors - query_vec, axis=1)
        distances = np.dot(index_vectors, query_vec)
        ann_lists.append(list(np.argsort(distances)[:k]))
    return np.array(ann_lists)

def compute_recall_at_k(
        nn_gt: np.ndarray,
        ann: np.ndarray,
        k: int,
):
    """
    This function computes the recall@k.
    Args:
        nn_gt: The ground truth nearest neighbors.
        ann: The approximate nearest neighbors.
        k: The number of nearest neighbors to consider.
    Returns:
        The recall@k.
    """
    return round(sum([len(set(ann[i]) & set(nn_gt[i])) / k for i in range(len(ann))])/len(ann), 3)

## Faiss Index

### Build Index

In [5]:
def build_faiss_lsh_index(
        index_vectors: np.ndarray,
        dim: int,
        nbits: int,
):
    """
    This function builds a Faiss LSH index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
        nbits: The number of bits to use in the hash.
    Returns:
        A Faiss LSH index.
    """
    index = faiss.IndexLSH(dim, nbits)
    index.add(index_vectors)
    return index

def build_faiss_flatl2_index(
        index_vectors: np.ndarray,
        dim: int,
):
    """
    This function builds a Faiss flat L2 index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
    Returns:
        A Faiss flat L2 index.
    """
    np.random.seed(42)
    num_vectors = index_vectors.shape[0]
    dim = index_vectors.shape[1]
    norm_index_vectors = index_vectors / np.linalg.norm(index_vectors, axis=1, keepdims=True)

    index = faiss.IndexFlatL2(dim)
    index.add(norm_index_vectors)
    return index

def faiss_search(
        query_vectors: np.ndarray,
        index: faiss.Index,
        k: int,
):
    """
    This function uses a Faiss index to search for the k-nearest neighbors of query_vectors.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        index: A Faiss index.
        k: The number of nearest neighbors to retrieve.
    Returns:
        An array of shape (, ) containing the indices of the k-nearest neighbors for each query vector.
    """
    if not isinstance(index, faiss.Index):
        raise ValueError("The index must be a Faiss index.")
    if isinstance(index, faiss.IndexFlatL2):
        num_queries = query_vectors.shape[0]
        dim = query_vectors.shape[1]
        query_vectors = query_vectors / np.linalg.norm(query_vectors, axis=1, keepdims=True)

    distances, indices = index.search(query_vectors, k)
    return indices

In [6]:
k=10
dim = index_vectors.shape[1]

In [63]:
%%time
faiss_lsh_index = build_faiss_lsh_index(index_vectors, dim, nbits=100)

In [7]:
%%time
faiss_l2_index = build_faiss_flatl2_index(index_vectors, dim)

CPU times: user 999 µs, sys: 1.27 ms, total: 2.27 ms
Wall time: 2.22 ms


### Compare Index Recall Compared to Exact Search

In [ ]:
### Exhaustive Search - Comparative Baseline ###
%%time
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)

In [ ]:
### Faiss LSH Search ###
faiss_lsh_ann = faiss_search(query_vectors, faiss_lsh_index, k)

print(f"recall@10 for faiss_lsh_index: {compute_recall_at_k(gt_nn, faiss_lsh_ann, k)}")

In [9]:
### Faiss L2 Search ###
faiss_l2_ann = faiss_search(query_vectors, faiss_l2_index, k)

#  print(f"recall@10 for faiss_l2_index: {compute_recall_at_k(gt_nn, faiss_l2_ann, k)}")

: 

: 

### Conclusions

For vectors resulting from CLIP embeddings, we see the best performance in ANN compared to exact search in the -- Index. With a recall of --.

We will use this index in the rest of out work and analysis.

## NMSLIB Index

In [1]:
%conda install -c conda-forge nmslib

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /Users/naomi/miniconda3/envs/Lab2_env

  added / updated specs:
    - nmslib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.8.30  |       hf0a4a13_0         155 KB  conda-forge
    certifi-2024.7.4           |     pyhd8ed1ab_0         156 KB  conda-forge
    nmslib-2.1.1               |  py310hb6aeb05_0         592 KB
    pybind11-2.13.5            |  py310h7306fd8_0         189 KB  conda-forge
    pybind11-global-2.13.5     |  py310h7306fd8_0         177 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.2 MB

The following NEW packages will be INSTALLED:

  nmslib             pkgs/main/osx-arm64::nmslib-2.1.1-py310hb6aeb05_0 
  pybind11           c

In [15]:
import nmslib

# Example data
data = np.random.random((1000, 128)).astype(np.float32)

# Create an index using a custom distance function
def custom_distance(vec1, vec2):
    return np.sum(np.abs(vec1 - vec2))  # Example: Manhattan distance

index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(data)
index.createIndex({'post': 2}, print_progress=True)

# Query
ids, distances = index.knnQuery(data[0], k=10)



0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

## PyNNDescent Index

In [ ]:
### run this in Lab2_env:
# conda install -c conda-forge pynndescent

In [5]:
import pynndescent
from numba import njit

In [6]:
### Define a unique distance function and compile it with Numba:
@njit
def custom_distance(x, y):
    return np.sum(np.abs(x - y))

In [8]:
index = pynndescent.NNDescent(index_vectors, metric=custom_distance)

In [9]:
indices, distances = index.query(query_vectors, k=5)

### Experiments with Cosine Similarity

In [35]:
k=20

In [41]:
### Exhaustive Search - Comparative Baseline ###
start = time.time()
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)
end = time.time()
print(f"Exhaustive search time: {end - start:.2f} seconds")

100%|██████████| 174/174 [00:00<00:00, 4104.34it/s]

Exhaustive search time: 0.04 seconds


In [37]:
start = time.time()
nndescent_index = pynndescent.NNDescent(index_vectors, metric='dot')
end = time.time()
print(f"Indexing time: {end - start:.2f} seconds")

Indexing time: 0.30 seconds


In [38]:
start = time.time()
indices, distances = nndescent_index.query(query_vectors, k)
end = time.time()
print(f"Query time: {end - start:.2f} seconds")

Query time: 0.88 seconds


In [42]:
recall = compute_recall_at_k(gt_nn, indices, k)
print(f"recall@{k} for pynndescent: {recall}")

recall@20 for pynndescent: 0.02
